<a href="https://colab.research.google.com/github/zivK12/Assignment3_API/blob/main/Assignment3_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment3_API
## Ziv Kelmer


In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
import requests

In [3]:
path = '/content/drive/MyDrive/data/'
fname = 'dests.txt'
with open(path+fname, 'r') as file:
  destinations = file.read().splitlines()
api_key='API_KEY'

In [12]:
def destination_info(destination, api_key):
    base_url = 'https://maps.googleapis.com/maps/api'
    distance_matrix_url = f'{base_url}/distancematrix/json'
    geocode_url = f'{base_url}/geocode/json'

    try:
        params = {
            'origins': 'Tel Aviv',
            'destinations': destination,
            'key': api_key
        }
        distance_data = requests.get(distance_matrix_url, params=params).json()
        distance = distance_data['rows'][0]['elements'][0]['distance']['text']
        duration = distance_data['rows'][0]['elements'][0]['duration']['text']


        params = {
            'address': destination,
            'key': api_key
        }
        geocode_data = requests.get(geocode_url, params=params).json()
        location = geocode_data['results'][0]['geometry']['location']
        longitude = location['lng']
        latitude = location['lat']

        return distance, duration, longitude, latitude

    except (requests.exceptions.RequestException, KeyError, IndexError):
        return None, None, None, None

data = {'Target': [], 'Distance_km': [], 'Duration': [], 'Longitude': [], 'Latitude': []}

for destination in destinations:
    distance, duration, longitude, latitude = destination_info(destination, api_key)
    distance_km = distance.split()[0]  
    data['Target'].append(destination)
    data['Distance_km'].append(distance_km)
    data['Duration'].append(duration)
    data['Longitude'].append(longitude)
    data['Latitude'].append(latitude)




df = pd.DataFrame(data)
df




,Target,Distance_km,Duration,Longitude,Latitude
0,Istanbul,"1,815",21 hours 3 mins,28.978359,41.008238
1,Amsterdam,"4,533",2 days 0 hours,4.904139,52.367573
2,Valletta,"3,793",2 days 3 hours,14.514100,35.899237
3,Basel,"4,118",1 day 20 hours,7.588576,47.559599
4,Doha,"2,164",22 hours 39 mins,51.531040,25.285447


In [24]:
sorted_cities = df.sort_values('Distance_km', ascending=False)
sorted_cities.reset_index(drop=True, inplace=True)
furthest_cities = sorted_cities['Target'].head(3)
furthest_cities


0    Amsterdam
1        Basel
2     Valletta
Name: Target, dtype: object